<a href="https://colab.research.google.com/github/napsugark/LLM_Course/blob/main/01_01_LLM_Learning_Path_2_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Prompt Engineering**

Link to older version: https://colab.research.google.com/drive/1wDqGnN_O8YfyMknQ4ArERsNiaLwea6B-

# Concepts to know


At the end of this module you should have an understanding of the following concepts:

- System Prompt
- Chain of Thought
- Few-Shot-Prompting
- Prompt Chaining
- Prompt Caching
- Temperature
- Top P
- Presence/Frequency penalty
- Structured Output
- Function calling


# Materials


### Mandatory

Introduction course to Prompt Engineering:

- https://www.coursera.org/projects/chatgpt-prompt-engineering-for-developers-project#outcomes"

Prompting guide (go through the Introduction Section and explore some of the Prompting Techniques):

- https://www.promptingguide.ai/

Prompt Caching:
- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/prompt-caching


### Optional

General resources on prompting:

- https://fvivas.com/en/19-formulas-and-prompt-structures-for-chatgpt/
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering?tabs=chat


OpenAI Parameters:

- https://www.prompthub.us/blog/understanding-openai-parameters-how-to-optimize-your-prompts-for-better-outputs

Azure OpenAI APIs:

- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling
- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure

# Azure Playground Assignments



You will complete the assignments in this sections in the Azure Playground. This is a UI layer around the Azure OpenAI endpoints that let's you build and experiment quickly with simple setups.

For the assignments 1-5 you will need to use Playgrounds - Chat Section, for the last assignment move to the Playgrounds Assistants Section.

Go to the [Azure Playground](https://ai.azure.com/resource/playground?wsid=/subscriptions/6f636525-4ca3-4447-841d-59b771d713b5/resourceGroups/MLResearch/providers/Microsoft.CognitiveServices/accounts/llm-learning-path&tid=39affebe-f048-4c77-ba6b-90cd8be22d25&deploymentId=/subscriptions/6f636525-4ca3-4447-841d-59b771d713b5/resourceGroups/MLResearch/providers/Microsoft.CognitiveServices/accounts/llm-learning-path/deployments/gpt-4o-mini). If you have any problems accessing the playground, please let your mentors know.



To help you get started in the playground and its functionalities, you can refer to this [guide](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line%2Ckeyless%2Ctypescript-keyless%2Cpython-new&pivots=programming-language-studio).

1. Define a chatbot that is only answering questions related to tourist attractions in your favourite holiday destinations. For any off-topic questions he should reply with "I don't know about that". Then, try to trick him into answering other questions.


2. Introduce a variable in the system prompt to change the response into any language.


3. Build a Summarizer for product reviews, that given a review text summarizes it in only one sentence. Try it with reviews with any product you like/are interested in.


4. Build a sentiment classifier, that given a product reviews returns a label for the sentiment of the review out of positive, negative, neutral.

5. Below you are given a text that summarizes a meeting between a care worker and his client. You are also given a list of existing goals for that client. Your task is to define a system prompt that attributes all the information contained in the transcript to the most accurate goal or, if there is no matching goal, attributes it to a general note. The model should respond in the JSON_format below and should not ommit any information.

In [ ]:
# Meeting Summary
meeting_summary = """
Over the past two weeks, I have visited the patient several times.
He is very variable in his behavior. His mood changes from day to day.
He indicate that he has slept poorly and often stays up late. He says it is because he worries a lot about his mother.
The patient clearly indicates that he does not want to go to the rehab clinic. He has found support with his brother and has the alcohol addiction under control.
He claims he keeps paying his bills. However, his mother indicates he is in high debt.
"""
# List of goals
list_of_goals = [
    "I can manage my administration.",
    "I have a healthy day and night rhythm",
    "I am able to discuss my emotions",
    "I clean my environment/home weekly",
    "I have appropriate help with my addiction"
]
json_format = {
    "goal_related_notes":[{"goal_name":"<name of goal chosen>",
                           "note":"<note attributed to that goal>"}],
    "general_notes":""}

6. In the assistant section of the playground, define a chatbot that given an adequate input will either call a tool to get the current weather or the current traffic for a given location. Use the weather template tool and adjust it for the traffic case.


In [ ]:
{
  "name": "get_weather",
  "description": "Determine weather in my location",
  "parameters": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": "The city and state e.g. Seattle, WA"
      },
      "unit": {
        "type": "string",
        "enum": [
          "c",
          "f"
        ]
      }
    },
    "required": [
      "location"
    ]
  }
}

# Coding

The following examples and assignments can be run inside your copy of the colab notebook.

Install the following to run the coding examples below

In [ ]:
pip install openai

## Example: Building a chatbot with the Azure Python SDK

In [ ]:
### Calling an LLM with Python Azure SDK
import os
from openai import AzureOpenAI
from google.colab import userdata

client = AzureOpenAI(
    azure_endpoint = userdata.get('AZURE_OPENAI_ENDPOINT'),
    api_key=userdata.get('AZURE_OPENAI_API_KEY'),
    api_version="2024-02-01" )

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
    ]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages= messages )

In [ ]:
# Inspect the response item
response

In [ ]:
# Extract the response string
response.choices[0].message.content


## Example: Building a Chatbot with memory


In [ ]:
class Chatbot:
    def __init__(self):
        self.client = AzureOpenAI(
            azure_endpoint=userdata.get('AZURE_OPENAI_ENDPOINT'),
            api_key=userdata.get('AZURE_OPENAI_API_KEY'),
            api_version="2024-02-01"
        )
        self.messages = [
            {"role": "system", "content": "You are a helpful assistant."}
        ]

    def get_response(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.messages
        )
        chatbot_response = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": chatbot_response})
        return chatbot_response

    def reset_conversation(self):
        self.messages = [
            {"role": "system", "content": "You are a helpful assistant."}
        ]


In [ ]:
chatbot = Chatbot()
chatbot.get_response("What is the capital of France?")

In [ ]:
chatbot.get_response("and how many people live there?")

## Assignment: Make a chatbot that can be set to either respond in English or Romanian.

## Assignment: Using structured outputs

Modify the Chatbot() Class from above such that given an input query of natural language that describes the topic, time and participant of a meeting, it returns a JSON containing the name, the datetime in ISO-8601, a list of the participants and an autmatically generated agenda. Use a Pydantic Model with descriptions to define the output format.

*Make sure to change the api_version when initializing your AzureOpenAI client to "2024-08-01" and use self.client.beta.chat.completions.parse() for generating the response*

## Example: Building a chatbot with chainlit interface


For this last section of the module we will be using Chainlit ([Docs](https://docs.chainlit.io/get-started/overview)). Chainlit is an open-source Python framework for data scientists and AI/ML engineers to deliver apps with a simple UI in few lines of code. We will be using chainlit throughout the course to have a simple interface for the applications you are working on.

To run the examples and work on the assignments you will need to create a local python repository and copy the code/files as instructed below:



Make sure you have Python installed on your system. You can follow this guide to setup Python for use in VS Code: https://code.visualstudio.com/docs/python/python-tutorial

- Via terminal make a venv with python -m .venv venv
- Via terminal activate your venv
 - on Windows with .venv\Scripts\activate
 - on macOs & Linux source .venv/bin/activate




- Make a .env file

In [ ]:
AZURE_OPENAI_ENDPOINT=<insert endpoint>
AZURE_OPENAI_API_KEY=<insert key>

- Make a requirements.txt file

In [ ]:
chainlit==2.5.5
openai==1.86.0
python-dotenv

- Install the requirements via terminal with pip install -r requirements.txt
- Copy the content below in an app.py file
- via terminal execute chainlit run app.py -w

In [ ]:
from openai import AzureOpenAI
import chainlit as cl
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

@cl.on_chat_start
async def start_chat():
    cl.user_session.set("client", AzureOpenAI(
        azure_endpoint=os.environ.get('AZURE_OPENAI_ENDPOINT'),
        api_key=os.environ.get('AZURE_OPENAI_API_KEY'),
        api_version="2024-06-01"
    ))
    cl.user_session.set("chat_history", [])


@cl.on_message
async def handle_message(message: cl.Message):

    msg = cl.Message(content="")

    chat_history = cl.user_session.get("chat_history", [])
    chat_history.append({"role": "user", "content": f"QUESTION: {message.content}"})

    response = cl.user_session.get("client").chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": m["role"], "content": m["content"]}
            for m in chat_history
        ],
        stream=True
    )

    for chunk in response:
        if not chunk.choices or not chunk.choices[0].delta:
            continue
        delta = chunk.choices[0].delta.content or ""
        await msg.stream_token(delta)

    # Update chat history
    chat_history.append({"role": "user", "content": message.content})
    chat_history.append({"role": "assistant", "content": msg.content})
    cl.user_session.set("chat_history", chat_history)
    await msg.update()

## Assignments Chainlit Chatbot:


Consult the chainlit documentation to work on the following assignments: [Chainlit Documentation](https://docs.chainlit.io/get-started/overview)

1. Add a System prompt specifying that the chatbot should always respond in Romanian.

2. Add a reset button to restart the conversation

*Hint: Use chainlit actions*

3. Add a setting option with a select field that allows you to specify the language of
the chatbot

*Hint: Use chainlit's Select input widget*


4. Add a Slider to the settings that sets the temperature

*Hint: Use chainlit's slider input widget*